In [1]:

from keys import FTX_API_KEY, FTX_API_SECRET
from datetime import datetime
import time
import hmac
import pandas as pd
from requests import Request, Session, Response
from typing import Optional, Dict, Any, List
from utility_classes.historical_data_processor import FTXDataProcessor




## getting all futures tickers

In [2]:
import json

acc = FTXDataProcessor(api_key=FTX_API_KEY, api_secret=FTX_API_SECRET)
current_futures = acc.get_all_futures_tickers()
# res = acc.send_request('GET', 'markets/BTC-PERP/candles?resolution=15&limit=10')
# all_trade = acc._get_all_trades('BTC-PERP', start_time='1559881511', end_time='1609477200')
# df = pd.DataFrame(res)
for i in current_futures:
    print(i)


1INCH-PERP
1INCH-0326
1INCH-0625
AAPL-0326
AAPL-0625
AAVE-PERP
AAVE-0326
AAVE-0625
ABNB-0326
ABNB-0625
ACB-0326
ACB-0625
ADA-PERP
ADA-0326
ADA-0625
ALGO-PERP
ALGO-0326
ALGO-0625
ALPHA-PERP
ALT-PERP
ALT-0326
ALT-0625
AMC-0326
AMC-0625
AMD-0326
AMD-0625
AMPL-PERP
AMZN-0326
AMZN-0625
APHA-0326
APHA-0625
AR-PERP
ARKK-0326
ARKK-0625
ATOM-PERP
ATOM-0326
ATOM-0625
AVAX-PERP
AVAX-0326
AVAX-0625
BABA-0326
BABA-0625
BADGER-PERP
BAL-PERP
BAL-0326
BAL-0625
BAND-PERP
BAO-PERP
BAT-PERP
BB-0326
BB-0625
BCH-PERP
BCH-0326
BCH-0625
BILI-0326
BILI-0625
BITW-0326
BITW-0625
BNB-PERP
BNB-0326
BNB-0625
BNT-PERP
BNTX-0326
BNTX-0625
BRZ-PERP
BRZ-0326
BRZ-0625
BSV-PERP
BSV-0326
BSV-0625
BTC-PERP
BTC-MOVE-0321
BTC-MOVE-0322
BTC-0326
BTC-MOVE-2021Q1
BTC-MOVE-WK-0326
BTC-MOVE-WK-0402
BTC-MOVE-WK-0409
BTC-MOVE-WK-0416
BTC-0625
BTC-MOVE-2021Q2
BTC-0924
BTC-MOVE-2021Q3
BTC-1231
BTC-HASH-2021Q1
BTMX-PERP
BTMX-0326
BTMX-0625
BTT-PERP
BYND-0326
BYND-0625
CAKE-PERP
CEL-0625
CGC-0326
CGC-0625
CHZ-PERP
CHZ-0326
CHZ-0625
CO

## getting all expired futures

In [3]:
expired_futures = acc._get('expired_futures')
for ticker in expired_futures: 
    print(ticker['name'], ticker['expiryDescription'])


mber 2020
AMZN-1225 December 2020
UBER-1225 December 2020
NVDA-1225 December 2020
EOS-1225 December 2020
BTMX-1225 December 2020
LINK-1225 December 2020
SOL-1225 December 2020
MID-1225 December 2020
DOT-1225 December 2020
DMG-1225 December 2020
BCH-1225 December 2020
SUSHI-1225 December 2020
TWTR-1225 December 2020
BNB-1225 December 2020
TLRY-1225 December 2020
SPY-1225 December 2020
BABA-1225 December 2020
ATOM-1225 December 2020
MATIC-1225 December 2020
OKB-1225 December 2020
BAL-1225 December 2020
SQ-1225 December 2020
TOMO-1225 December 2020
MRNA-1225 December 2020
BSV-1225 December 2020
RUNE-1225 December 2020
ABNB-1225 December 2020
CREAM-1225 December 2020
SHIT-1225 December 2020
GOOGL-1225 December 2020
BTC-HASH-2020Q4 2020 Q4
DRGN-1225 December 2020
USDT-1225 December 2020
TRYB-1225 December 2020
BTC-1225 December 2020
BTC-MOVE-1224 2020-12-24
BTC-MOVE-1223 2020-12-23
BTC-MOVE-1222 2020-12-22
BTC-MOVE-1221 2020-12-21
BTC-MOVE-1220 2020-12-20
BTC-MOVE-1219 2020-12-19
BTC-MOVE-1

## all past futures expiration dates
### to get past data requires correct date input 

In [4]:
acc.get_expired_futures_dates()


expiration date: 1225 | Date December 2020
expiration date: 0925 | Date September 2020
expiration date: 0626 | Date June 2020
expiration date: 20200327 | Date March 2020
expiration date: 20191227 | Date December 2019
expiration date: 20190927 | Date September 2019
expiration date: 20190628 | Date June 2019
expiration date: 20190329 | Date March 2019


## specific expired futures data 

In [8]:
eth_hist_futures = acc.get_expired_futures_OHCL(market = 'ETH-1225', year = 2020, resolution = 3600)
print(eth_hist_futures)

Adding 2506 candles with start time 2020-12-25 00:00:00
Adding 1 candles with start time 2020-09-11 14:00:00
                      timestamp     open    high      low   close  \
0     2020-09-11T18:00:00+00:00  366.755  368.38  366.205  368.05   
1     2020-09-11T19:00:00+00:00  368.050  370.99  365.830  369.43   
2     2020-09-11T20:00:00+00:00  369.430  371.18  368.470  369.15   
3     2020-09-11T21:00:00+00:00  369.150  371.06  369.150  370.36   
4     2020-09-11T22:00:00+00:00  370.400  376.29  370.370  374.75   
...                         ...      ...     ...      ...     ...   
2501  2020-12-24T23:00:00+00:00  612.550  614.81  608.520  611.73   
2502  2020-12-25T00:00:00+00:00  611.730  615.41  607.540  611.34   
2503  2020-12-25T01:00:00+00:00  611.340  614.09  609.330  611.64   
2504  2020-12-25T02:00:00+00:00  611.640  612.04  608.800  609.59   
2505  2020-12-25T03:00:00+00:00  609.590  609.59  609.530  609.57   

            volume  next_open  
0      23398.11591     368.05 